**Device config:**

In [ ]:
import getpass
hostname = input("Device Hostname:")
username = input("Username:")
password = getpass.getpass()

# Configuring a device

There are two methods to configuring the device, structured and unstructured.

## Structured Configuration Changes

Structured configuration changes are useful when the user does not know configuration stanzas or how to configure a Junos device. It abstracts the configuration statements from the user and allows changes to be made through tables and views.

## Unstructured Configuration Changes

Unstructured configuration changes allow the user to make changes via "normal" configuration commands using XML, Junos set commands or JSON. These changes are performed using the ` jnpr.junos.utils.config.Config ` utility. 

After connecting to the device you create a `Config` object and associate it to the device:

    from jnpr.junos import Device
    from jnpr.junos.utils.config import Config

    dev = Device(host=hostname, user=username, password=password, port='22')
    cu = Config(dev)
    
The basic process for making changes is to:

1. Lock the configuarion using ` lock() `
2. Use `load()`, `rollback()` or `rescue()` to make changes
3. Commit the configuration using `commit()`
4. Unlock the configuration using `unlock()`


## Unstructured Example

*Note, I had to adjust the timeout on commit() to 120 seconds since my compute node is slow and the vSRX was taking >30 seconds (the default timeout) to commit.*

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config

dev = Device(host=hostname, user=username, password=password, port='22')

dev.open()

with Config(dev, mode='private') as cu:  
    cu.load('set system services netconf traceoptions file test.log', format='set')
    cu.pdiff()
    cu.commit(timeout=120)
    
    
dev.close()

## Rollback Example

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config

dev = Device(host=hostname, user=username, password=password, port='22')

dev.open()

with Config(dev, mode='private') as cu:  
    cu.rollback(1)
    cu.pdiff()
    cu.commit(timeout=120)
    
dev.close()

## Configuration Comparing

`pdiff()` and `diff()` can be used to print the difference between the current candidate configuration and current active configuration.

`pdiff()` is the same as "`show compare | rollback`" and will print the difference to stdout (console). `diff()` will return the difference as an object.

If you use pdiff() prior to making changes the output will be "None"

### `pdiff()` Example

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config

dev = Device(host=hostname, user=username, password=password, port='22')

dev.open()

with Config(dev, mode='private') as cu:
    cu.pdiff(2)
dev.close()

### `diff()` Example

In [ ]:
from jnpr.junos import Device
from jnpr.junos.utils.config import Config

dev = Device(host=hostname, user=username, password=password, port='22')

dev.open()

with Config(dev, mode='private') as cu:
    diff = cu.diff(5)
    print(diff)
dev.close()

### Comparing exceptions

Warning: If you attempt to load a configuration rollback and the RB ID does not exist a `RPCError` exception will be generated.